In [25]:
import pandas as pd
import os
import numpy as np
import re

In [17]:
unique_med=pd.read_excel(r"D:/vijay/gritsys-rasa/med_list.xlsx",usecols=["Med"])

In [23]:
unique_med.head(5)

,Med
0,Saridon
1,dolo650
2,gelusil
3,combiflame
4,Crocin


In [21]:
np.unique(unique_med).size

100

###  We have 100 unique medicines used in training examples for Entity Recognition model

###  Reading Training examples

In [28]:
#  Converting input data to a form that can be used for Annotating

with open(r"D:\vijay\gritsys-rasa\custom_examples.txt","r") as s:
    t=s.read()
    t=re.sub("]","",re.sub("\[","",t))
    t=re.sub("\(","",t)
    t=re.sub("\)","",t)
    t=re.sub(";","",t)
    t=re.sub("medicine_name","",t)

In [30]:
#  Saving training example with correct format

with open(r"spacy_input_examples/custom_examples_new_format.txt","w") as s:
    s.write(t)

In [34]:
# We can now Annotate the input example for Entity "Medicine_name"  using NER Entity Annotator - 
# [ https://tecoholic.github.io/ner-annotator/ ] and output with annotation be stored in  spacy_input_examples/  folder

In [35]:
#  Now we have our input data with Annotations for Entity. Let us start building the model

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

c:\Users\Vijay\anaconda3\envs\capestone_1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [37]:
#  Create  blank spacy model that can be trained on our data and create empty Docbin file 

nlp=spacy.blank("en")  # new   blank spacy model
db=DocBin()

In [40]:
annoted_inputs=  open(r"spacy_input_examples/input_medicine_annots.json","r")
annoted_inputs=json.load(annoted_inputs)

In [42]:
#   Input text with Annotations in JSON format
print(annoted_inputs)

{'classes': ['MEDICINE_NAME'], 'annotations': [['I wanted to know about  Saridon tablet\r', {'entities': [[24, 31, 'MEDICINE_NAME']]}], ['For what dolo650  tablet is used?\r', {'entities': [[9, 16, 'MEDICINE_NAME']]}], ['Help me understand the side effects and usage of gelusil drug?\r', {'entities': [[49, 56, 'MEDICINE_NAME']]}], ['Uses of combiflame syrup\r', {'entities': [[8, 18, 'MEDICINE_NAME']]}], ['Dosage instructions for Saridon?\r', {'entities': [[24, 31, 'MEDICINE_NAME']]}], ['May I know the usage of Saridon drug?\r', {'entities': [[24, 31, 'MEDICINE_NAME']]}], ['May I know the side effects of Saridon drug?\r', {'entities': [[31, 38, 'MEDICINE_NAME']]}], ['I need to understand the side effects of Crocin tablet\r', {'entities': [[41, 47, 'MEDICINE_NAME']]}], ['Would you help me understand about combiflame\r', {'entities': [[35, 45, 'MEDICINE_NAME']]}], ['Dosage information of Saridon\r', {'entities': [[22, 29, 'MEDICINE_NAME']]}], ['Interested in understanding effect of Combifl

In [43]:
#  Converting our input text to Spacy span object and adding it to Docbin object which we have created


for text, annot in tqdm(annoted_inputs['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    doc.ents = ents 
    db.add(doc)

# Store the docbin object to disk
db.to_disk("spacy_input_examples/training_data.spacy") # save the docbin object

100%|██████████| 117/117 [00:00<00:00, 1857.18it/s]


In [44]:
# creating Config file   -  This pipeline has English as Language and it is configured to run in CPU
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2023-01-15 22:55:27.412011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-01-15 22:55:27.412640: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
c:\Users\Vijay\anaconda3\envs\capestone_1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-01-15 22:55:33.953063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-01-15 22:55:33.953645: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-15 22:55:33.969432: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUD

In [45]:
#  Traing spacy model with the training data that we have created
! python -m spacy train config.cfg --output spacy_output/ --paths.train spacy_input_examples/training_data.spacy --paths.dev spacy_input_examples/training_data.spacy

ℹ Saving to output directory: spacy_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     46.50    0.00    0.00    0.00    0.00
 17     200         17.56   1118.80  100.00  100.00  100.00    1.00
 37     400          0.00      0.00  100.00  100.00  100.00    1.00
 63     600          0.00      0.00  100.00  100.00  100.00    1.00
 94     800          0.00      0.00  100.00  100.00  100.00    1.00
131    1000          0.00      0.00  100.00  100.00  100.00    1.00
178    1200          0.00      0.00  100.00  100.00  100.00    1.00
233    1400          0.00      0.00  100.00  100.00  100.00    1.00
299    1600          0.00

2023-01-15 22:57:53.870233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-01-15 22:57:53.871342: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
c:\Users\Vijay\anaconda3\envs\capestone_1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-01-15 22:58:03.236084: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-01-15 22:58:03.237220: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-15 22:58:03.266892: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUD

In [46]:
# Let us load the model which has been built and check

nlp_ner = spacy.load("spacy_output/model-best") 

In [54]:
outt=nlp_ner("I wanted to know the dosage info of Moxikind-CV Dry Syrup")

In [55]:
print("Actual text",outt)
print("Entity detected",outt.ents)
spacy.displacy.render(outt, style="ent", jupyter=True) 

Actual text I wanted to know the dosage info of Moxikind-CV Dry Syrup
Entity detected (Moxikind-CV Dry,)


##### Now at this stage we can load this model inside Action server in Rasa and we can pass text to get Entity. 
##### But inorder to integrate this model in Rasa pipeline and make Rasa to call this model when needed we need to package this model and install it in this environment 

In [57]:
#  Convert the best model to python package. This step will create the tar file

! python -m spacy package spacy_output/model-best . --force

ℹ Building package artifacts: sdist
✔ Loaded meta.json from file
spacy_output\model-best\meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory 'en_pipeline-0.0.0'
en_pipeline-0.0.0
running sdist
running egg_info
creating en_pipeline.egg-info
writing en_pipeline.egg-info\PKG-INFO
writing dependency_links to en_pipeline.egg-info\dependency_links.txt
writing entry points to en_pipeline.egg-info\entry_points.txt
writing requirements to en_pipeline.egg-info\requires.txt
writing top-level names to en_pipeline.egg-info\top_level.txt
writing manifest file 'en_pipeline.egg-info\SOURCES.txt'
reading manifest file 'en_pipeline.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'en_pipeline.egg-info\SOURCES.txt'
running check
creating en_pipeline-0.0.0
creating en_pipeline-0.0.0\en_pipeline
creating en_pipeline-0.0.0\en_pipeline.egg-info
creating en_pipeline-0.0.0\en_pipeline\en_pipeline-0.0.0
creating en_pipeline-0.0.0\en_pipeline

2023-01-15 23:10:26.586314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-01-15 23:10:26.587006: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
c:\Users\Vijay\anaconda3\envs\capestone_1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-01-15 23:10:35.483494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-01-15 23:10:35.484077: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-15 23:10:35.500192: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUD

In [58]:
#  Now let us install  the  package which we have created

! python -m pip  install en_pipeline-0.0.0/dist/custom_medicine_ner.tar.gz

Processing .\en_pipeline-0.0.0\dist\custom_medicine_ner.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en-pipeline: filename=en_pipeline-0.0.0-py3-none-any.whl size=5659979 sha256=23e63114170a7ba9a4c5ef8835ebc6c4d751ef769902c6b1f7796a09342ab3eb
  Stored in directory: C:\Users\Vijay\AppData\Local\Temp\pip-ephem-wheel-cache-_jjes4j5\wheels\c1\ff\0b\ee2360ecf39088aa11046388311aef1478e2bbffebf3ac830d
Successfully built en-pipeline



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


###  Now we have installed our custom Spacy NER model to this environment. We can use it in Rasa config files